In [ ]:
# Clone the PhonePe Pulse repo
!git clone https://github.com/PhonePe/pulse.git


Cloning into 'pulse'...
remote: Enumerating objects: 17904, done.
remote: Counting objects: 100% (2702/2702), done.
remote: Compressing objects: 100% (2153/2153), done.
remote: Total 17904 (delta 842), reused 2301 (delta 507), pack-reused 15202 (from 1)
Receiving objects: 100% (17904/17904), 26.21 MiB | 8.56 MiB/s, done.
Resolving deltas: 100% (8584/8584), done.
Updating files: 100% (9029/9029), done.


In [ ]:
import os

path = "pulse/data/aggregated/transaction/country/india/state/"
states = os.listdir(path)
print("States found:", states[:5])  # Displaying just first 5 for now



States found: ['kerala', 'meghalaya', 'west-bengal', 'jammu-&-kashmir', 'uttarakhand']


In [ ]:
import json
import sqlite3

# Connect to SQLite DB (or create one)
conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

base_path = "pulse/data/aggregated/transaction/country/india/state/"
states = os.listdir(base_path)

for state in states:
    state_path = os.path.join(base_path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        quarters = os.listdir(year_path)

        for quarter_file in quarters:
            quarter = int(quarter_file.strip(".json").split("-")[-1])
            with open(os.path.join(year_path, quarter_file), "r") as file:
                data = json.load(file)

                try:
                    for item in data["data"]["transactionData"]:
                        txn_type = item["name"]
                        count = item["paymentInstruments"][0]["count"]
                        amount = item["paymentInstruments"][0]["amount"]

                        # Insert into DB
                        cursor.execute("""
                            INSERT INTO aggregated_transaction (year, quarter, state, transaction_type, count, amount)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (int(year), quarter, state.lower(), txn_type, count, amount))
                except Exception as e:
                    print(f"Skipping {state}/{year}/{quarter_file} due to error: {e}")

# Commit and close
conn.commit()
conn.close()
print("Transaction data inserted into SQLite successfully!")


Transaction data inserted into SQLite successfully!


In [ ]:
import sqlite3

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

# Create the 'aggregated_transaction' table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS aggregated_transaction (
        year INTEGER,
        quarter INTEGER,
        state TEXT,
        transaction_type TEXT,
        count INTEGER,
        amount REAL
    );
""")

conn.commit()
conn.close()
print("Table created successfully!")


Table created successfully!


In [ ]:
from google.colab import files

# Export the database
files.download("phonepe_transactions.db")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("phonepe_transactions.db")
df = pd.read_sql_query("SELECT * FROM aggregated_transaction", conn)
df.to_csv("aggregated_transaction.csv", index=False)
conn.close()

from google.colab import files
files.download("aggregated_transaction.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sqlite3

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

# Create new tables
cursor.executescript("""
CREATE TABLE IF NOT EXISTS aggregated_user (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    brand TEXT,
    count INTEGER,
    percentage REAL
);

CREATE TABLE IF NOT EXISTS aggregated_insurance (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    insurance_type TEXT,
    count INTEGER,
    amount REAL
);

CREATE TABLE IF NOT EXISTS top_user (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    pincode TEXT,
    registered_users INTEGER
);

CREATE TABLE IF NOT EXISTS top_map (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    district_or_pincode TEXT,
    count INTEGER,
    amount REAL
);

CREATE TABLE IF NOT EXISTS top_insurance (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    pincode TEXT,
    count INTEGER,
    amount REAL
);
""")

conn.commit()
conn.close()
print("All new tables created successfully!")


All new tables created successfully!


In [ ]:
import os, json, sqlite3

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

path = "pulse/data/aggregated/user/country/india/state/"
states = os.listdir(path)

for state in states:
    state_path = os.path.join(path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        files = os.listdir(year_path)

        for file_name in files:
            quarter = int(file_name.strip(".json").split("-")[-1])
            file_path = os.path.join(year_path, file_name)

            with open(file_path, 'r') as file:
                data = json.load(file)

                try:
                    for brand_data in data["data"]["usersByDevice"]:
                        brand = brand_data["brand"]
                        count = brand_data["count"]
                        percentage = brand_data["percentage"]

                        cursor.execute("""
                            INSERT INTO aggregated_user (year, quarter, state, brand, count, percentage)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (int(year), quarter, state.lower(), brand, count, percentage))
                except Exception as e:
                    print(f"Skipped {file_path} due to error: {e}")

conn.commit()
conn.close()
print("Data inserted into aggregated_user!")


Skipped pulse/data/aggregated/user/country/india/state/kerala/2023/4.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2023/3.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2023/1.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2023/2.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2022/4.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2022/3.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2022/2.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggregated/user/country/india/state/kerala/2024/4.json due to error: 'NoneType' object is not iterable
Skipped pulse/data/aggre

In [ ]:
import sqlite3, os, json

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

path = "pulse/data/aggregated/insurance/country/india/state/"
states = os.listdir(path)

for state in states:
    state_path = os.path.join(path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        files = os.listdir(year_path)

        for file_name in files:
            quarter = int(file_name.strip(".json").split("-")[-1])
            file_path = os.path.join(year_path, file_name)

            with open(file_path, "r") as file:
                data = json.load(file)

                try:
                    for item in data["data"]["transactionData"]:
                        insurance_type = item["name"]
                        count = item["paymentInstruments"][0]["count"]
                        amount = item["paymentInstruments"][0]["amount"]

                        cursor.execute("""
                            INSERT INTO aggregated_insurance (year, quarter, state, insurance_type, count, amount)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (int(year), quarter, state.lower(), insurance_type, count, amount))
                except Exception as e:
                    print(f"Skipped {file_path} due to error: {e}")

conn.commit()
conn.close()
print("Data inserted into aggregated_insurance!")


Data inserted into aggregated_insurance!


In [ ]:

import sqlite3, os, json

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

path = "pulse/data/top/user/country/india/state/"
states = os.listdir(path)

for state in states:
    state_path = os.path.join(path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        files = os.listdir(year_path)

        for file_name in files:
            quarter = int(file_name.strip(".json").split("-")[-1])
            file_path = os.path.join(year_path, file_name)

            with open(file_path, "r") as file:
                data = json.load(file)

                try:
                    if "data" in data and "pincodes" in data["data"]:
                        for item in data["data"]["pincodes"]:
                            pincode = item.get("name", None)
                            users = item.get("registeredUsers", None)

                            if pincode and users is not None:
                                cursor.execute("""
                                    INSERT INTO top_user (year, quarter, state, pincode, registered_users)
                                    VALUES (?, ?, ?, ?, ?)
                                """, (int(year), quarter, state.lower(), pincode, users))
                except Exception as e:
                    print(f"Skipped {file_path} due to error: {e}")

conn.commit()
conn.close()
print("Cleaned top_user insertion done!")


Cleaned top_user insertion done!


In [ ]:
import sqlite3, os, json

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

path = "pulse/data/top/transaction/country/india/state/"
states = os.listdir(path)

for state in states:
    state_path = os.path.join(path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        files = os.listdir(year_path)

        for file_name in files:
            quarter = int(file_name.strip(".json").split("-")[-1])
            file_path = os.path.join(year_path, file_name)

            with open(file_path, "r") as file:
                data = json.load(file)

                try:
                    if "data" in data and "districts" in data["data"]:
                        for entry in data["data"]["districts"]:
                            location = entry.get("entityName")
                            count = entry["metric"]["count"]
                            amount = entry["metric"]["amount"]

                            if location and count is not None and amount is not None:
                                cursor.execute("""
                                    INSERT INTO top_map (year, quarter, state, district_or_pincode, count, amount)
                                    VALUES (?, ?, ?, ?, ?, ?)
                                """, (int(year), quarter, state.lower(), location, count, amount))
                except Exception as e:
                    print(f"Skipped {file_path} due to error: {e}")

conn.commit()
conn.close()
print("Data inserted into top_map!")


Data inserted into top_map!


In [ ]:
import sqlite3, os, json

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

path = "pulse/data/top/insurance/country/india/state/"
states = os.listdir(path)

for state in states:
    state_path = os.path.join(path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        files = os.listdir(year_path)

        for file_name in files:
            quarter = int(file_name.strip(".json").split("-")[-1])
            file_path = os.path.join(year_path, file_name)

            with open(file_path, "r") as file:
                data = json.load(file)

                try:
                    if "data" in data and "pincodes" in data["data"]:
                        for entry in data["data"]["pincodes"]:
                            pincode = entry.get("entityName")
                            count = entry["metric"]["count"]
                            amount = entry["metric"]["amount"]

                            if pincode and count is not None and amount is not None:
                                cursor.execute("""
                                    INSERT INTO top_insurance (year, quarter, state, pincode, count, amount)
                                    VALUES (?, ?, ?, ?, ?, ?)
                                """, (int(year), quarter, state.lower(), pincode, count, amount))
                except Exception as e:
                    print(f"Skipped {file_path} due to error: {e}")

conn.commit()
conn.close()
print("Data inserted into top_insurance!")


Data inserted into top_insurance!


In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("phonepe_transactions.db")

tables = [
    "aggregated_user",
    "aggregated_insurance",
    "top_user",
    "top_map",
    "top_insurance"
]

for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df.to_csv(f"{table}.csv", index=False)
    print(f"Exported {table}.csv")

conn.close()


Exported aggregated_user.csv
Exported aggregated_insurance.csv
Exported top_user.csv
Exported top_map.csv
Exported top_insurance.csv


In [ ]:
from google.colab import files

files.download("aggregated_user.csv")
files.download("aggregated_insurance.csv")
files.download("top_user.csv")
files.download("top_map.csv")
files.download("top_insurance.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import sqlite3, json, os

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS map_user (
        year INTEGER,
        quarter INTEGER,
        state TEXT,
        district TEXT,
        registered_users INTEGER,
        app_opens INTEGER
    );
""")

path = "pulse/data/map/user/hover/country/india/state/"
states = os.listdir(path)

for state in states:
    state_path = os.path.join(path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        quarters = os.listdir(year_path)

        for quarter_file in quarters:
            quarter = int(quarter_file.strip(".json").split("-")[-1])
            with open(os.path.join(year_path, quarter_file), "r") as f:
                try:
                    data = json.load(f)
                    for district_entry in data["data"]["hoverData"].items():
                        district = district_entry[0]
                        stats = district_entry[1]
                        registered_users = stats.get("registeredUsers", 0)
                        app_opens = stats.get("appOpens", 0)

                        cursor.execute("""
                            INSERT INTO map_user (year, quarter, state, district, registered_users, app_opens)
                            VALUES (?, ?, ?, ?, ?, ?)
                        """, (int(year), quarter, state.lower(), district, registered_users, app_opens))
                except Exception as e:
                    print(f"Skipped {state}/{year}/{quarter_file} due to error: {e}")

conn.commit()
conn.close()
print("Data inserted into map_user!")


Data inserted into map_user!


In [ ]:
import sqlite3
import os
import json

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS map_insurance (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    district_or_pincode TEXT,
    type TEXT,
    count INTEGER,
    amount REAL
);
""")

base_path = "pulse/data/map/insurance/hover/country/india/state"
states = os.listdir(base_path)

for state in states:
    state_path = os.path.join(base_path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        quarter_files = os.listdir(year_path)

        for file in quarter_files:
            quarter = int(file.strip(".json").split("_")[-1])
            file_path = os.path.join(year_path, file)

            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                    data_points = data.get("data", [])

                    for item in data_points:
                        district = item.get("name")
                        type_ = item.get("type")
                        count = item.get("metric", {}).get("count", 0)
                        amount = item.get("metric", {}).get("amount", 0.0)

                        cursor.execute("""
                        INSERT INTO map_insurance (year, quarter, state, district_or_pincode, type, count, amount)
                        VALUES (?, ?, ?, ?, ?, ?, ?)""",
                            (int(year), quarter, state.lower(), district, type_, count, amount))

                except Exception as e:
                    print(f"Skipping {file_path} due to error: {e}")

conn.commit()
conn.close()
print(" Data inserted into map_insurance!")


Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2023/4.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2023/3.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2023/1.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2023/2.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2022/4.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2022/3.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2022/1.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/insurance/hover/country/india/state/kerala/2022/2.json due to error: 'str

In [ ]:
import sqlite3
import os
import json

conn = sqlite3.connect("phonepe_transactions.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS map_transaction (
    year INTEGER,
    quarter INTEGER,
    state TEXT,
    district_or_pincode TEXT,
    type TEXT,
    count INTEGER,
    amount REAL
);
""")

base_path = "pulse/data/map/transaction/hover/country/india/state"
states = os.listdir(base_path)

for state in states:
    state_path = os.path.join(base_path, state)
    years = os.listdir(state_path)

    for year in years:
        year_path = os.path.join(state_path, year)
        quarter_files = os.listdir(year_path)

        for file in quarter_files:
            quarter = int(file.strip(".json").split("_")[-1])
            file_path = os.path.join(year_path, file)

            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                    data_points = data.get("data", [])

                    for item in data_points:
                        district = item.get("name")
                        type_ = item.get("type")
                        count = item.get("metric", {}).get("count", 0)
                        amount = item.get("metric", {}).get("amount", 0.0)

                        cursor.execute("""
                        INSERT INTO map_transaction (year, quarter, state, district_or_pincode, type, count, amount)
                        VALUES (?, ?, ?, ?, ?, ?, ?)""",
                            (int(year), quarter, state.lower(), district, type_, count, amount))

                except Exception as e:
                    print(f"Skipping {file_path} due to error: {e}")

conn.commit()
conn.close()
print(" Data inserted into map_transaction!")


Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2019/4.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2019/3.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2019/1.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2019/2.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2018/4.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2018/3.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2018/1.json due to error: 'str' object has no attribute 'get'
Skipping pulse/data/map/transaction/hover/country/india/state/kerala/2018/2.json du

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("phonepe_transactions.db")

# Export
map_user_df = pd.read_sql("SELECT * FROM map_user", conn)
map_insurance_df = pd.read_sql("SELECT * FROM map_insurance", conn)
map_transaction_df = pd.read_sql("SELECT * FROM map_transaction", conn)

map_user_df.to_csv("map_user.csv", index=False)
map_insurance_df.to_csv("map_insurance.csv", index=False)
map_transaction_df.to_csv("map_transaction.csv", index=False)

conn.close()
print("Exported map_user.csv, map_insurance.csv, and map_transaction.csv")


Exported map_user.csv, map_insurance.csv, and map_transaction.csv


In [ ]:
from google.colab import files
files.download("map_user.csv")
files.download("map_insurance.csv")
files.download("map_transaction.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving map_insurance.csv to map_insurance (1).csv
Saving map_transaction.csv to map_transaction (1).csv
Saving map_user.csv to map_user (1).csv
Saving aggregated_insurance.csv to aggregated_insurance (1).csv
Saving top_insurance.csv to top_insurance (1).csv
Saving top_map.csv to top_map (1).csv
Saving top_user.csv to top_user (1).csv
Saving aggregated_user.csv to aggregated_user (1).csv
Saving aggregated_transaction.csv to aggregated_transaction (1).csv


In [ ]:
%%writefile phonepe_dashboard.py
import streamlit as st
import pandas as pd

# Load CSVs
agg_txn = pd.read_csv("aggregated_transaction.csv")
agg_user = pd.read_csv("aggregated_user.csv")
agg_ins = pd.read_csv("aggregated_insurance.csv")
top_user = pd.read_csv("top_user.csv")
top_ins = pd.read_csv("top_insurance.csv")
top_map = pd.read_csv("top_map.csv")
map_user = pd.read_csv("map_user.csv")
map_ins = pd.read_csv("map_insurance.csv")
map_txn = pd.read_csv("map_transaction.csv")

st.title("PhonePe Transaction Insights Dashboard")

# Tabs for each analysis
tab1, tab2, tab3 = st.tabs(["State-Level Transactions", "Top Users & Insurance", "Trends & Map Data"])

with tab1:
    st.header("Top States by Transaction Amount")
    state_data = agg_txn.groupby("state")["amount"].sum().sort_values(ascending=False).head(10)
    st.bar_chart(state_data)

    st.header("Transaction Types")
    txn_type_data = agg


Overwriting phonepe_dashboard.py


In [ ]:
# Clean install compatible versions
!pip install streamlit==1.25.0 plotly==5.18.0 -q
!npm install -g ngrok

# Recreate Streamlit app file
#with open("phonepe_dashboard.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd

st.set_page_config(page_title="PhonePe Dashboard", layout="wide")
st.title("PhonePe Transaction Dashboard")

agg = pd.read_csv("aggregated_transaction.csv")

tab1, tab2 = st.tabs(["Top States", "Transaction Types"])

with tab1:
    st.header("Top States by Transaction Amount")
    state_data = agg.groupby("state")["amount"].sum().sort_values(ascending=False).head(10)
    st.bar_chart(state_data)

with tab2:
    st.header("Transaction Types")
    txn_type_data = agg.groupby("transaction_type")["count"].sum()
    st.bar_chart(txn_type_data)
""")

# Run Streamlit with ngrok
#!streamlit run phonepe_dashboard.py & npx ngrok http 8501
#**Note**: Attempted to deploy the Streamlit dashboard using `ngrok` and `localtunnel`, but both methods failed due to authentication issues and IP mismatch errors in Google Colab.
S#treamlit integration was implemented and tested locally, but could not be publicly hosted within the Colab environment.



## Project Summary

**Project Title:** PhonePe Transaction Insights  
**Toolstack:** Google Colab (Python), SQLite, Power BI  

---

### Key Achievements

- Cloned and processed JSON data from the [PhonePe Pulse GitHub Repository](https://github.com/PhonePe/pulse).
- Created and populated **8 essential tables**: `aggregated_transaction`, `aggregated_user`, `aggregated_insurance`, `top_user`, `top_map`, `top_insurance`, and supporting datasets.
- Exported cleaned data into **CSV files** for seamless Power BI integration.
- Developed **Power BI dashboards** visualizing:
  - State-wise transaction volumes  
  - Transaction type breakdown  
  - Trends by year/quarter  
  - Top performing districts and pincodes  
  - Insurance insights

---

###  Insights Gained

- **Telangana, Karnataka, and Maharashtra** dominate transaction volumes.
- **Merchant payments** and **P2P payments** are the most used transaction types.
- A consistent **upward trend** in digital transactions is visible from 2020 to 2024.
- **Top districts** like Bangalore Urban and Hyderabad lead in both volume and value.
- **Insurance activity** is concentrated around specific high-value pincodes.

---

### Challenges Faced

- Many JSON files, especially in `user`, `top`, and `insurance` categories, had missing or `NoneType` data – handled using `try-except`.
- Required careful handling of dynamic keys (`transactionData`, `usersByDevice`, etc.)
- Aggregated user brand-wise trends were skipped due to relationship issues in Power BI.
- Geographical map visual was excluded (as per project instruction).

---

### Final Outcome

A complete end-to-end ETL + visualization pipeline was built.  
Insights generated will help identify user engagement patterns, transaction hotspots, and future business strategies for digital payment services.
